In [10]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def count_num_price_changed(df):

    num_price_changed = 0
    last_price = df["last"][0]
    for i in range(len(df)):
        if df["last"][i] != last_price:
            num_price_changed += 1
            last_price = df["last"][i]
            
    return num_price_changed
        


In [12]:
dfx = pd.DataFrame(columns=['diff_h_l_0',
                            'diff_o_c_0',
                            'diff_h_o_0',
                            'diff_c_l_0',
                            'vol_total_0',
                            'vol_buy_0',
                            'vol_sell_0',
                            'vol_other_0',
                            'nb_tick_total_0',
                            'nb_tick_buy_0',
                            'nb_tick_sell_0',
                            'nb_tick_other_0',
                            'nb_price_changed_0',
                            'nb_tick_buy_vol_eq_1',
                            'nb_tick_sell_vol_eq_1',
                            'nb_tick_other_vol_eq_1',
                            'nb_tick_buy_vol_2_5',
                            'nb_tick_sell_vol_2_5',
                            'nb_tick_other_vol_2_5'
                            'diff_open_low_m1'
                           ])


In [13]:
for filename in os.scandir("data_mes"):
    if filename.is_file():
        
        #print(filename.path)
        df = pd.read_csv(filename.path, sep=";")
        # print(len(df))

        t = df["time_msc"][0]
        t = int(str(t)[:-3])
        t = t*1000
        t = t+60000
        df2 = df[df["time_msc"] < t]
        df2 = df2.reset_index()
        df1 = df[df["time_msc"].between(t, t+60000)]
        df1 = df1.reset_index()
        df0 = df[df["time_msc"] > t+60000]
        df0 = df0.reset_index()
        
        new_row = pd.Series({
                            'diff_h_l_0': df0['last'].max() - df0['last'].min(),
                            'diff_o_c_0': df0['last'][0] - df0['last'][len(df0)-1],
                            'diff_h_o_0': df0['last'].max() - df0['last'][0],
                            'diff_c_l_0': df0['last'][len(df0)-1] - df0['last'].min(),                            
                            'vol_total_0': df0['volume'].sum(),
                            'vol_buy_0': df0.loc[df0['buy'] == 1, 'volume'].sum(),
                            'vol_sell_0': df0.loc[df0['sell'] == 1, 'volume'].sum(),
                            'vol_other_0': df0.loc[(df0['buy'] == 0) & (df0['sell'] == 0), 'volume'].sum(),                            
                            'nb_tick_total_0': df0['last'].count(),
                            'nb_tick_buy_0': df0.loc[df0['buy'] == 1, 'last'].count(),
                            'nb_tick_sell_0': df0.loc[df0['sell'] == 1, 'last'].count(),
                            'nb_tick_other_0': df0.loc[(df0['buy'] == 0) & (df0['sell'] == 0), 'last'].count(),                            
                            'nb_price_changed_0': count_num_price_changed(df0),                            
                            'nb_tick_buy_vol_eq_1': df0.loc[(df0['buy'] == 1) & (df0['volume'] == 1), 'last'].count(),
                            'nb_tick_sell_vol_eq_1': df0.loc[(df0['sell'] == 1) & (df0['volume'] == 1), 'last'].count(),
                            'nb_tick_other_vol_eq_1': df0.loc[(df0['buy'] == 0) & (df0['sell'] == 0) & (df0['volume'] == 1), 'last'].count(),
                            'nb_tick_buy_vol_2_5': df0.loc[(df0['buy'] == 1) & (df0['volume'].between(2,5)), 'last'].count(),
                            'nb_tick_sell_vol_2_5': df0.loc[(df0['sell'] == 1) & (df0['volume'].between(2,5)), 'last'].count(),
                            'nb_tick_other_vol_2_5': df0.loc[(df0['buy'] == 0) & (df0['sell'] == 0) & (df0['volume'].between(2,5)), 'last'].count(),                            
                            'diff_open_low_m1': df0.loc[0]['open_m1'] - df0.loc[0]['low_m1'],
                            })
        
        #print(new_row)
        
        dfx = pd.concat([dfx, new_row.to_frame().T], ignore_index=True)
        


In [14]:
dfx.to_csv("_prod_df_m1_lower_.csv", sep=";", index=False)

In [9]:
dfx

,diff_h_l_0,diff_o_c_0,diff_h_o_0,diff_c_l_0,vol_total_0,vol_buy_0,vol_sell_0,vol_other_0,nb_tick_total_0,nb_tick_buy_0,...,nb_tick_other_0,nb_price_changed_0,nb_tick_buy_vol_eq_1,nb_tick_sell_vol_eq_1,nb_tick_other_vol_eq_1,nb_tick_buy_vol_2_5,nb_tick_sell_vol_2_5,nb_tick_other_vol_2_5diff_open_low_m1,nb_tick_other_vol_2_5,diff_open_low_m1
0,8.50,3.50,0.50,4.50,14533.0,5557.0,7811.0,1212.0,10488.0,4111.0,...,1029.0,722.0,3462.0,4459.0,945.0,594.0,857.0,NaN,77.0,0.75
1,8.25,5.50,0.00,2.75,11150.0,4090.0,6029.0,1107.0,8479.0,3235.0,...,955.0,539.0,2799.0,3587.0,864.0,409.0,716.0,NaN,89.0,4.00
2,5.75,4.00,1.50,0.25,8409.0,3419.0,4362.0,645.0,6144.0,2445.0,...,550.0,308.0,2044.0,2685.0,495.0,368.0,427.0,NaN,55.0,5.00
3,5.25,3.50,0.00,1.75,11844.0,5102.0,5867.0,906.0,8535.0,3650.0,...,681.0,540.0,3069.0,3567.0,598.0,501.0,593.0,NaN,75.0,1.00
4,5.75,4.25,0.50,1.00,8937.0,3693.0,4428.0,848.0,6421.0,2604.0,...,667.0,321.0,2153.0,2672.0,595.0,411.0,464.0,NaN,65.0,3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,2.50,1.25,0.75,0.50,3499.0,1550.0,1697.0,275.0,2280.0,944.0,...,233.0,171.0,740.0,862.0,211.0,182.0,225.0,NaN,22.0,0.50
380,1.50,0.75,0.75,0.00,2555.0,1044.0,1299.0,216.0,1688.0,689.0,...,163.0,99.0,547.0,657.0,140.0,124.0,158.0,NaN,22.0,0.25
381,1.00,0.25,0.75,0.00,1485.0,619.0,747.0,120.0,919.0,357.0,...,84.0,55.0,271.0,394.0,70.0,70.0,72.0,NaN,12.0,0.25
382,2.50,2.25,0.25,0.00,3530.0,1358.0,1897.0,295.0,2206.0,834.0,...,190.0,157.0,631.0,936.0,149.0,173.0,212.0,NaN,34.0,1.50
